# Reacher Arm Project

![Screenshot of reacher arm environment](doc/BannerImage.png)

This is an implementation of the Deep Deterministic Policy Gradients Algorithm for training a two joint arm to keep its end effector within a moving target volume.

## Table of Contents
+ Environment Setup
+ Description of Algorithm
  - Value Distribution
  - Parameterized Model
  - Prioritized Replay
+ Implementation of Algorithm
  - Hyperparameters
  - Prioritized Replay Buffer
  - Action Value Distribution Function (Neural Network)
  - Bellman Update (Loss) Computation
  - Training Loop
+ Training
+ Results
+ References

## Environment Setup

+ Follow instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to set up the environment, *with the following changes:*
  - Before running `pip install .`, edit `Value-based-methods/python/requirements.txt` and remove the `torch==0.4.0` line
  - After running `pip install .`, run the appropriate PyTorch installation command for your system indicated [here](https://pytorch.org/get-started/locally/)
  - Continue following the instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to their conclusion.
+ Download the appropriate Unity Environment for your platform:
  - [Linux](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Linux.zip)
  - [Mac OSX](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher.app.zip)
  - [Windows (32-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86.zip)
  - [Windows (64-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86_64.zip)
+ Place the Unity Environment zip file into any convenient directory, and unzip the file.

### Imports and references
Run the following code cell at every kernel instance start-up to bring implementation dependencies into the notebook namespace, and identify the path to the simulated environment executable.

In [ ]:
from unityagents import UnityEnvironment
from collections import namedtuple, deque
from math import isnan
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# Set to the path to simulated environment executable on system.
env_location = \
"C:\Projects\UdacityRLp2\Reacher_Windows_x86_64\Reacher_Windows_x86_64\Reacher.exe"

## Description of Algorithm

The Deep Deterministic Policy Gradient (DDPG) algorithm extends the application of Q-Learning methods to action spaces with continuously valued dimensions [[1]](#References).  There are two networks involved, a Policy Network and Action Value (Q) network.  During learning, the Policy Network generates an action according to the input state, and both this action and the state are supplied to the Q network as input.  The Q network outputs a single action value, and the gradient of this value with respect to the parameters of the Policy Network are used to nudge the policy towards one with a higher action value (by gradient ascent). <br><br> 
In typical Deep Q Learning (DQN), each action has a corresponding output from the Q network, but this representation of the Action Value Function $Q(s,a)$ does not naturally accomodate continuously-valued actions.  The primary difference of DDPG with respect to DQN, is that actions are instead explicit, continuously valued inputs to the Action Value function approximation.  This allows closed-form computation of the gradient of $Q(s,a)$ with respect to changes in magnitudes of the continuously-valued action variables.

The basic DDPG algorithm reads as follows (from [[1]](#References)): <br>
![DDPG Algorithm](doc/DDPG_alg.png) <br>

This implementation incorporates the following improvements:

### N-Step Bootstrapping

The hyperparameter `n_step_order`, determines the value of $n$ in the following alternative Bellman Update target, replacing the definition for $y_i$ in the algorithm above:

$$y_i = r_i + \gamma r_{i+1} + \gamma^2 r_{i+2} + ... + \gamma^n r_{i+n} + \gamma^{n+1} Q'(s_{i+n+1},\mu '(s_{i+n+1}|\theta^{\mu '})|\theta^{Q'})$$

N-Step Bootstrapping increases the relative weight of sampled rewards from the environment, compared to rewards estimated by the Action Value function $Q(s,a)$. Anecdotally, this seems to assist action values propagating backwards in time and through 'bottlenecks' where most nearby states have comparatively low State Values.  So essentially, initial learning can be faster, and some connections may be made that would otherwise take an unacceptably long time to be made without N-Step Bootstrapping.  However, real rewards are stochastic, and an atypically bad or good run of events will  more readily propagate through a Q network with N-Step Bootstrapping.  If an agent quickly changes its behavior between simple, regimented approaches, it is possible the `n_step_order` value in use is too high for the agent's environment.

### Prioritized Replay
The algorithm will periodically switch between exploration and learning phases.  <br><br>During exploration phases, state transition tuples $(S_t,a_t,r_t,S_{t+1})$ will be collected, transformed to *n-step* transition events via an accumulation buffer, and stored in a prioritized experience buffer. 
<br><br>
During learning phases, transition events sampled from the prioritized experience buffer will be used to optimize the parameters of the Policy and Q networks.  Like in [[2]](#References), the probability of utilizing a transition $T$ from the experience buffer is consistent with the proportionality relation: <br><br>
$$p_T \varpropto (Loss)^{\alpha}, \alpha \in [0,\infty)$$
<br>
The hyperparameter $\alpha$ allows tuning of the degree to which the probability of selection is affected by loss magnitude [[2]](#References).
<br><br>Qualitatively, the $Loss$ in this context is proportional to how inconsistent the parameterized model's prediction is with a prediction that uses actual rewards sampled from the environment.  See the implementation section for detail on how the loss is computed.

## References
[1] Lillicrap et. al., Continuous control with deep reinforcement learning, [arXiv:1509.02971](https://arxiv.org/abs/1509.02971) <br>
[2] Schaul et. al., Prioritized Experience Replay, [arXiv:1511.05952](https://arxiv.org/abs/1511.05952)<br>
[4] http://www.sefidian.com/2022/09/09/sumtree-data-structure-for-prioritized-experience-replay-per-explained-with-python-code/<br>
[5] Fortunato et. al., Noisy Networks for Exploration, arXiv:1706.10295